In [2]:
import numpy as np
import h5py

with h5py.File("/home/ljian/private/PHYS 139 Final Project/deepclean-1251335314-4097.h5", "r") as f:
    X = []
    for channel, timeseries in f.items():
        if channel == "H1:GDS-CALIB_STRAIN":
            y = timeseries[:]
        else:
            X.append(timeseries[:])

X = np.stack(X, axis=-1)

print(X.shape)
print(y.shape)

(16781312, 21)
(16781312,)


In [3]:
train_max_index = int(X.shape[0] * 4 / 5)

X_train = X[:train_max_index]
y_train = y[:train_max_index]

X_test = X[train_max_index:]
y_test = y[train_max_index:]

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(13425049, 21)
(13425049,)
(3356263, 21)
(3356263,)


In [4]:
window_length = 2 # s
sample_rate = 4096 # Hz
window_size = window_length * sample_rate

# train_batches in your code is currently 1000, which is fine, 
# but i set it here to int(total time steps / window size).
train_batches = int(X_train.shape[0]/window_size) 
X_train_batch, y_train_batch = [], []
for i in range(train_batches):
    idx = np.random.randint(X_train.shape[0] - window_size)
    X_train_batch.append(X_train[idx: idx + window_size])
    y_train_batch.append(y_train[idx: idx + window_size])

X_train_batch = np.stack(X_train_batch)
y_train_batch = np.stack(y_train_batch)

print(X_train_batch.shape)
print(y_train_batch.shape)

(1638, 8192, 21)
(1638, 8192)


In [ ]:
# the idea here is the test data should be fixed and not randomized/shuffled
# so we're just going to grab as many contiguous slices of it as we can
test_batches = int(X_test.shape[0]/window_size)
X_test_batch, y_test_batch = [], []
for i in range(test_batches):
    idx = i * window_size
    X_test_batch.append(X_test[idx: idx + window_size])
    y_test_batch.append(y_test[idx: idx + window_size])

X_test_batch = np.stack(X_test_batch)
y_test_batch = np.stack(y_test_batch)

print(X_test_batch.shape)
print(y_test_batch.shape)

In [ ]:
def send_train_test():
    """
    strain_mean == mean of data used for z-score
    strain_std = std dev used for z-score
    
    Will be needed in post processing. Call function to return the old mean and std dev to reverse z-score
    """
    return X_train_batch, y_train_batch, X_test_batch, y_test_batch